In [1]:
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import MobileNet, MobileNetV2, VGG19, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.optimize import minimize
import tensorflow as tf

2025-05-21 09:10:21.672381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747818621.849575      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747818621.900311      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load all image paths
all_paths = glob.glob('/kaggle/input/hg14-handgesture14-dataset/HG14/HG14-Hand Gesture/*/*.jpg')

# Prepare dataset
data, labels = [], []
for path in tqdm(all_paths):
    img = cv2.imread(path)
    img = cv2.resize(img, (128, 128))
    label = path.split('/')[-2]
    data.append(img)
    labels.append(label)

data = np.array(data) / 255.0
labels = pd.factorize(labels)[0]

# Split data
X_temp, X_test, y_temp, y_test = train_test_split(data, labels, test_size=0.10, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.20, stratify=y_temp, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

100%|██████████| 14000/14000 [02:02<00:00, 114.39it/s]
/tmp/ipykernel_35/213992276.py:14: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels = pd.factorize(labels)[0]


In [3]:
# Model Builder
def build_model(base_model_fn, input_shape=(128, 128, 3), num_classes=14):
    base_model = base_model_fn(include_top=False, input_shape=input_shape, weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False  # Freeze base layers initially

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# Settings
batch_size = 20
epochs = 50

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3)
]

models_dict = {
    'MobileNetV2': MobileNetV2,
    'VGG19': VGG19
}

trained_models = {}
val_preds = {}
test_preds = {}



In [5]:
from keras.callbacks import ModelCheckpoint

trained_models = {}
val_preds = {}
test_preds = {}

for name, model_fn in models_dict.items():
    print(f"\nTraining {name}")
    
    model = build_model(model_fn)

    # Create a checkpoint callback specific to this model
    checkpoint_cb = ModelCheckpoint(
        filepath=f"/kaggle/working/{name}_best.h5",
        save_best_only=True,
        monitor="val_accuracy",   # or "val_loss"
        mode="max",               # "min" if using val_loss
        verbose=1
    )

    model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint_cb],  # using model-specific callback
        verbose=1
    )

    val_preds[name] = model.predict(X_val)
    test_preds[name] = model.predict(X_test)
    trained_models[name] = model



Training MobileNetV2


I0000 00:00:1747819456.905621      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747819456.906235      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1747819470.222936     100 service.cc:148] XLA service 0x7e7d14003790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747819470.223889     100 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747819470.223909     100 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747819471.164195     100 cuda_dnn.cc:529] Loaded cuDNN version 90300


  4/504 ━━━━━━━━━━━━━━━━━━━━ 27s 54ms/step - accuracy: 0.1188 - loss: 3.7131 

I0000 00:00:1747819475.643032     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


504/504 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6166 - loss: 1.2264
Epoch 1: val_accuracy improved from -inf to 0.90476, saving model to /kaggle/working/MobileNetV2_best.h5
504/504 ━━━━━━━━━━━━━━━━━━━━ 57s 86ms/step - accuracy: 0.6168 - loss: 1.2255 - val_accuracy: 0.9048 - val_loss: 0.2632
Epoch 2/50
504/504 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.8232 - loss: 0.4960
Epoch 2: val_accuracy improved from 0.90476 to 0.91905, saving model to /kaggle/working/MobileNetV2_best.h5
504/504 ━━━━━━━━━━━━━━━━━━━━ 35s 69ms/step - accuracy: 0.8232 - loss: 0.4959 - val_accuracy: 0.9190 - val_loss: 0.2329
Epoch 3/50
504/504 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.8653 - loss: 0.3754
Epoch 3: val_accuracy improved from 0.91905 to 0.92937, saving model to /kaggle/working/MobileNetV2_best.h5
504/504 ━━━━━━━━━━━━━━━━━━━━ 35s 69ms/step - accuracy: 0.8653 - loss: 0.3754 - val_accuracy: 0.9294 - val_loss: 0.1936
Epoch 4/50
504/504 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.8674 

In [19]:
# Ensemble
val_stack = np.stack([val_preds[name] for name in models_dict.keys()], axis=-1)

def dirichlet_loss(weights):
    ensemble_pred = np.tensordot(val_stack, weights, axes=([2], [0]))
    return -np.mean(np.sum(to_categorical(y_val, 14) * np.log(ensemble_pred + 1e-8), axis=1))

init_weights = np.ones(len(models_dict)) / len(models_dict)
bounds = [(0, 1)] * len(models_dict)
constraints = [{'type': 'eq', 'fun': lambda w: 1 - sum(w)}]

res = minimize(dirichlet_loss, init_weights, bounds=bounds, constraints=constraints)
final_weights = res.x
print("Optimized Weights:", final_weights)

# Final Test Prediction
test_stack = np.stack([test_preds[name] for name in models_dict.keys()], axis=-1)
ensemble_test_pred = np.tensordot(test_stack, final_weights, axes=([2], [0]))
ensemble_test_labels = np.argmax(ensemble_test_pred, axis=1)

print("Ensemble Accuracy on Test Set:", accuracy_score(y_test, ensemble_test_labels))


Optimized Weights: [0.95019789 0.04980211]
Ensemble Accuracy on Test Set: 0.9828571428571429


In [20]:
import numpy as np
from tensorflow.keras.models import load_model

class EnsembleModel:
    def __init__(self, model_paths, weights):
        self.models = [load_model(p) for p in model_paths]
        self.weights = np.array(weights)

    def predict(self, X):
        preds = [model.predict(X) for model in self.models]
        stacked = np.stack(preds, axis=-1)
        ensemble_pred = np.tensordot(stacked, self.weights, axes=([2], [0]))
        return ensemble_pred

    def predict_class(self, X):
        return np.argmax(self.predict(X), axis=1)